# Hermes — Run a basic Hartree-Fock energy calculation

See [the tutorial](/Tutorials/hermes_hartree_fock_energy_calculation.ipynb) for an in-depth explanation of how the Hermes Hartree-Fock energy calculation works.

In [ ]:
# |hide
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/hermes_hartree_fock_quickstart/").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR)
os.chdir(WORK_DIR)

In [ ]:
# Get a pdb to work with
# We use the pdb-tools cli here, but you can download directly from rcsb.org
!pdb_fetch '1brs' | pdb_selchain -A | pdb_delhetatm > '1B39_A_nohet.pdb'
!ls

1B39_A_nohet.pdb


In [ ]:
import json
from pathlib import Path

import rush

client = rush.build_blocking_provider_with_functions(
    batch_tags=["hermes_quickstart"],
)

prepared_protein_qdxf, prepared_protein_pdb = client.prepare_protein(
    client.workspace / "1B39_A_nohet.pdb"
)

# There may be multiple conformers, so select the first one
(first_conformer,) = client.pick_conformer(prepared_protein_qdxf, 0)

# Fragment the protein so that we can run a quantum energy calculation
(fragmented_protein,) = client.fragment_aa(first_conformer, 1, "All", 30)

fragmented_protein_contents = json.loads(
    fragmented_protein.download().read_text()
)

# Quantum energy calculation
(hermes_energy,) = client.hermes_energy(
    fragmented_protein,
    {},
    {
        "basis": "STO-3G",
        "aux_basis": "6-31G",
        "method": "RestrictedHF",
    },  # configuration for a fast converging, low accuracy run
    None,
    {
        "level": "Dimer",
        "cutoffs": {"dimer": 15},
        "reference_fragment": len(
            fragmented_protein_contents["topology"]["fragments"]
        )
        - 1,
    },
    None,
    resources={"gpus": 1, "storage": 100, "storage_units": "MB", "walltime": 60},
)

energy = json.load(open(hermes_energy.download(), "r"))
# Let's look at the expanded Hartree Fock energy
energy["expanded_hf_energy"]

2024-03-19 14:51:09,990 - rush - INFO - Restoring by default via env
2024-03-19 14:51:11,205 - rush - INFO - Trying to restore job with tags: ['hermes_quickstart'] and path: github:talo/prepare_protein/947cdbc000031e192153a20a9b4a8fbb12279102#prepare_protein_tengu
2024-03-19 14:51:11,377 - rush - INFO - Trying to restore job with tags: ['hermes_quickstart'] and path: github:talo/tengu-prelude/b345a0b0077225c63d904d0e03fb9ca1acec55ed#pick_conformer
2024-03-19 14:51:11,511 - rush - INFO - Trying to restore job with tags: ['hermes_quickstart'] and path: github:talo/tengu-prelude/4a16cf12be35f0a40c113d4410046de865f1906f#fragment_aa
2024-03-19 14:51:11,783 - rush - INFO - Argument b7f9b99c-6af4-45f4-bd73-3bf84603c05f is now ModuleInstanceStatus.RESOLVING
2024-03-19 14:53:04,633 - rush - INFO - Argument b7f9b99c-6af4-45f4-bd73-3bf84603c05f is now ModuleInstanceStatus.ADMITTED
2024-03-19 14:53:14,698 - rush - INFO - Argument b7f9b99c-6af4-45f4-bd73-3bf84603c05f is now ModuleInstanceStatus.DIS

0.5644136911942041